In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport

# Load the dataset
df = pd.read_csv("melb_data.csv")

In [ ]:
# Task 1: Handling Missing Values
print("Step 1: Checking and Handling Missing Data")
missing_ratio = df.isnull().sum() / len(df) * 100
cols_to_remove = missing_ratio[missing_ratio > 20].index
df.drop(columns=cols_to_remove, inplace=True)
print(f"Columns removed due to excessive missing values: {list(cols_to_remove)}")

# Fill missing values
numeric_cols = df.select_dtypes(include=['number'])
df.fillna(numeric_cols.median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])
print("Filled missing values using median for numbers and mode for categorical data.")

In [ ]:
# Task 2: Filtering & Indexing
print("\nStep 2: Extracting and Filtering Data")
if {"Suburb", "Price"}.issubset(df.columns):
    richmond_listings = df[(df["Suburb"] == "Richmond") & (df["Price"] > 1_000_000)]
    print(f"Number of Richmond properties priced above $1M: {richmond_listings.shape[0]}")
else:
    richmond_listings = pd.DataFrame()

essential_cols = {"Landsize", "Price", "Suburb", "BuildingArea"}
if essential_cols.issubset(df.columns):
    large_land_props = df.loc[df["Landsize"] > 500, ["Price", "Suburb", "BuildingArea"]]
    print(f"Total properties with land size exceeding 500 sqm: {len(large_land_props)}")
else:
    large_land_props = pd.DataFrame()
    print("Skipping land size filtering due to missing columns.")

if "Price" in df.columns:
    expensive_props = df.sort_values(by="Price", ascending=False).head(5)
    print("Top 5 priciest properties:")
    print(expensive_props)
else:
    expensive_props = pd.DataFrame()
    print("Unable to rank properties by price due to missing data.")

In [ ]:
# Task 3: Data Transformation
print("\nStep 3: Transforming Data for Insights")
if {"Price", "Rooms"}.issubset(df.columns):
    df["Cost_per_Room"] = df["Price"] / df["Rooms"]
    print("New column 'Cost_per_Room' created.")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    df.dropna(subset=["Date"], inplace=True)
    df["Sold_Year"] = df["Date"].dt.year
    print("Converted 'Date' column and extracted year of sale.")

In [ ]:
# Task 4: Grouping & Summarization
print("\nStep 4: Aggregation and Summarization")
if {"Suburb", "Price"}.issubset(df.columns):
    avg_price_per_suburb = df.groupby("Suburb")["Price"].mean()
    total_listings_per_suburb = df.groupby("Suburb")["Price"].count()
    highest_priced_suburb = avg_price_per_suburb.idxmax()
    print(f"The suburb with the highest average price: {highest_priced_suburb}")
else:
    avg_price_per_suburb = pd.Series(dtype=float)
    total_listings_per_suburb = pd.Series(dtype=int)
    highest_priced_suburb = "Data Unavailable"

In [ ]:
# Task 5: Data Export & Reporting
print("\nStep 5: Saving Processed Data")
df.to_csv("cleaned_dataset.csv", index=False)
profile = ProfileReport(df, title="Automated Data Report", explorative=True)
profile.to_file("data_analysis_report.html")
print("Generated report and saved cleaned dataset.")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Bonus Task: Advanced Insights
print("\nBonus: Additional Analysis")
if "Price" in df.columns:
    price_median = df["Price"].median()
    df["Price_Category"] = df["Price"].apply(lambda p: "Expensive" if p > price_median else "Affordable")
    price_room_correlation = df["Price"].corr(df["Rooms"]) if "Rooms" in df.columns else "N/A"
    print(f"Correlation between price and rooms: {price_room_correlation}")
else:
    price_room_correlation = "N/A"

In [ ]:
# Save Summary
with open("summary_report.txt", "w") as f:
    f.write(f"Suburb with highest price: {highest_priced_suburb}\n")
    f.write(f"Correlation between price & rooms: {price_room_correlation}\n")
